In [1]:
from fetch_census_data import acs5_aggregate, acs5_individual, acs5_profile
from fetch_fcc_data import fcc_fixed

In [4]:
data = acs5_profile()

In [5]:
print(data)

      DP02_0001E  DP02_0152E  DP02_0152PE  DP02_0153E  DP02_0153PE  \
0            575         426         74.1         392         68.2   
1           1544        1411         91.4        1242         80.4   
2           1172        1068         91.1         917         78.2   
3           1084        1003         92.5         917         84.6   
4            574         356         62.0         322         56.1   
...          ...         ...          ...         ...          ...   
1314        1980        1556         78.6        1125         56.8   
1315        1665        1326         79.6        1071         64.3   
1316        1597        1260         78.9        1088         68.1   
1317        1154         959         83.1         888         76.9   
1318        1109         945         85.2         899         81.1   

      DP03_0062E  DP03_0062PE  DP03_0119E  DP03_0119PE  DP05_0001E  \
0          37422   -888888888  -888888888         25.7        1825   
1          47000   

In [ ]:
fcc_fixed()